In [ ]:
# 3d-plots Lat-Lon-time, Lat-Lon-depth
# pia goecke
# 29.11.20224

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
import os


In [2]:
%matplotlib qt

In [3]:
#read data and rename cols
os.chdir('C:/Users/piago/Documents/Uni/5_Semester/Hiwi')
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE614_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file, skiprows=10)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df.rename(columns={'CTD_Temperature_SML' : 'CTD_Temperature_1',
                   'CTD_Temperature_30cm' : 'CTD_Temperature_30',
                   'CTD_Temperature_40cm' : 'CTD_Temperature_40',
                   'CTD_Temperature_50cm_[°C]' : 'CTD_Temperature_50',
                   'CTD_Temperature_60cm_[°C]': 'CTD_Temperature_60',
                   'CTD_Temperature_85cm' : 'CTD_Temperature_85',
                   'CTD_Temperature_100cm' : 'CTD_Temperature_100',
                   'RBR_Temperature_30cm' :  'RBR_Temperature_30',
                   'RBR_Temperature_40cm' :  'RBR_Temperature_40',
                   'RBR_Temperature_50cm' :  'RBR_Temperature_50',
                   'RBR_Temperature_60cm' :  'RBR_Temperature_60',
                   'RBR_Temperature_85cm' :  'RBR_Temperature_85',
                   'RBR_Temperature_100cm' :  'RBR_Temperature_100'
                   }, inplace= True)

df['Date'] = df.index.date
df_backup = df

In [4]:
# extract dfs
df_2 = df.loc['2023-03-02']
df_2 = df_2[(df_2.index < '2023-03-02 09:48') | (df_2.index >= '2023-03-02 09:49')]

df_3 = df.loc['2023-03-03']

df_8 = df.loc['2023-03-08']
df_8 = df_8[(df_8.index < '2023-03-08 09:19:10') | (df_8.index >= '2023-03-08 09:21:40')]


df_9 = df.loc['2023-03-09']
df_9.drop(df_9[df_9['CTD_Temperature_1']>6.2].index, inplace = True)


df_10 = df.loc['2023-03-10']

df_11 = df.loc['2023-03-11']

df_17 = df.loc['2023-03-17']
df_17 = df_17[df_17.index > '2023-03-17 08:03']

df_18 = df.loc['2023-03-18']
df_18 = df_18[df_18.index > '2023-03-18 07:50']

df_19 = df.loc['2023-03-19']
df_19 = df_19[(df_19.index < '2023-03-08 10:15:30') | (df_19.index >= '2023-03-19 10:16:20')]

df_20 = df.loc['2023-03-20']


all_df = [df_2, df_3, df_8, df_9, df_10, df_11, df_17, df_18, df_19, df_20]

In [5]:
#Detrend CTD, RBR and Campbell Air temp where reasonable

for df in [df_2, df_9, df_17]:
    # time in total seconds
    df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

    # fit polynomial with numpy.polyfit:  quadratic
    coefficients = np.polyfit(df['time_numeric'], df['CTD_Temperature_1'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_SML'] = polynomial(df['time_numeric']) #values
    df[f'detrend_2_Temp_SML'] = df['CTD_Temperature_1'] - df[f'fitted_2_Temp_SML']
    df[f'detrend_2_Temp_SML'] = df[f'detrend_2_Temp_SML'] + df[f'fitted_2_Temp_SML'].median()

    # same for all depths
    depths = [30, 40, 50, 60, 85, 100]

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"Column CTD_Temperature_{d}cm_[°C] contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()


    # ... and same for airtemp
    coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
    df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
    df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

    # ... and for RBRs
    depths = [30, 40, 50, 60, 85, 100]
    for d in depths:
        if df[f'RBR_Temperature_{d}'].dropna().empty:
            print(f"Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
        else: 
            coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

    print(f'mission {str(df['Date'][0])} detrended.')

Column CTD_Temperature_40cm_[°C] contains only NaN values. Creating nan-column for depth 40.
mission 2023-03-02 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-09 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-17 detrended.


In [6]:
df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]

In [7]:
df = df_3

In [8]:
sensor = "CTD"  # set to "CTD" or "RBR"


if sensor == "RBR":
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended RBR data is used')
        df = df.reset_index()
        df = df[['Latitude', 'Longitude', 'detrend_2_Temp_30', 'detrend_2_Temp_40','detrend_2_Temp_50',  'detrend_2_Temp_60', 'detrend_2_Temp_85', 'detrend_2_Temp_100' ]]

        df.rename(columns={
                        'rbr_detrend_2_Temp_30' : 'RBR_Temperature_30',
                        'rbr_detrend_2_Temp_40' : 'RBR_Temperature_40',
                        'rbr_detrend_2_Temp_50' : 'RBR_Temperature_50',
                        'rbr_detrend_2_Temp_60': 'RBR_Temperature_60',
                        'rbr_detrend_2_Temp_85' : 'RBR_Temperature_85',
                        'rbr_detrend_2_Temp_100' : 'RBR_Temperature_100'}, inplace= True)
    else:
        print('original RBR data is used.')
        df = df.reset_index()
        df = df[['Latitude', 'Longitude', 'RBR_Temperature_30', 
                'RBR_Temperature_40', 
                'RBR_Temperature_50',  
                'RBR_Temperature_60', 'RBR_Temperature_85', 'RBR_Temperature_100' ]]


    # Melt the DataFrame: Reshape from wide to long format
    df_melted = df.melt(id_vars=['Latitude', 'Longitude'], 
                        value_vars=['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50',  
                                    'RBR_Temperature_60', 'RBR_Temperature_85', 'RBR_Temperature_100' ], 
                        var_name='Depth', 
                        value_name='Temperature')

    # Extract the depth information from the column names
    df_melted['Depth'] = (df_melted['Depth'].str.extract(r'(\d+)$').astype(float))
    # Now df_melted has columns: Latitude, Longitude, Temperature, and Depth
    df = df_melted
    df.head()

elif sensor == 'CTD':
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended CTD data is used')
        df = df.reset_index()
        df = df[['Latitude', 'Longitude', 'detrend_2_Temp_SML', 'detrend_2_Temp_30', 'detrend_2_Temp_40','detrend_2_Temp_50',  'detrend_2_Temp_60', 'detrend_2_Temp_85', 'detrend_2_Temp_100' ]]

        df.rename(columns={'detrend_2_Temp_SML' : 'CTD_Temperature_1',
                        'detrend_2_Temp_30' : 'CTD_Temperature_30',
                        'detrend_2_Temp_40' : 'CTD_Temperature_40',
                        'detrend_2_Temp_50' : 'CTD_Temperature_50',
                        'detrend_2_Temp_60': 'CTD_Temperature_60',
                        'detrend_2_Temp_85' : 'CTD_Temperature_85',
                        'detrend_2_Temp_100' : 'CTD_Temperature_100'}, inplace= True)
    else:
        print('original CTD data is used.')
        df = df.reset_index()
        df = df[['Latitude', 'Longitude', 'CTD_Temperature_1', 'CTD_Temperature_30', 
                'CTD_Temperature_40', 
                'CTD_Temperature_50',  
                'CTD_Temperature_60', 'CTD_Temperature_85', 'CTD_Temperature_100' ]]


    # Melt the DataFrame: Reshape from wide to long format
    df_melted = df.melt(id_vars=['Latitude', 'Longitude'], 
                        value_vars=['CTD_Temperature_1', 'CTD_Temperature_30','CTD_Temperature_40',
                                    'CTD_Temperature_50',
                                    'CTD_Temperature_60', 'CTD_Temperature_85', 'CTD_Temperature_100' ], 
                        var_name='Depth', 
                        value_name='Temperature')

    # Extract the depth information from the column names
    df_melted['Depth'] = (df_melted['Depth'].str.extract(r'(\d+)$').astype(float))
    # Now df_melted has columns: Latitude, Longitude, Temperature, and Depth
    df = df_melted
    df.head()

else:
    print('That did not work. Please enter valid sensor. "CTD" or "RBR"')

original CTD data is used.


In [9]:
# drop lines where Temperature is na
df = df.dropna(subset = ['Temperature'])

# Extract the columns for Latitude, Longitude, Depth, and Temperature
latitudes = df['Latitude']
longitudes = df['Longitude']
depths = df['Depth']
temperature = df['Temperature']


In [10]:

# Define a grid for Latitude, Longitude, and Depth
# adjust these ranges and resolution depending on your data and plot needs
grid_longitudes = np.linspace(longitudes.min(), longitudes.max(), 20) 

grid_latitudes = np.linspace(latitudes.min(), latitudes.max(), 8)  

grid_depths = np.linspace(depths.min(), depths.max(), 10) # 10 points in depth direction

# Create the meshgrid for the grid of Latitude, Longitude, and Depth
grid_lon, grid_lat, grid_depth = np.meshgrid(grid_longitudes, grid_latitudes, grid_depths) # negative depths


In [11]:

# Flatten the grids for interpolation (scipy.griddata requires 1D arrays for input)
points = np.array([longitudes, latitudes, depths]).T  # Original scattered data points
grid_points = np.array([grid_lon.flatten(), grid_lat.flatten(), grid_depth.flatten()]).T  # Grid points


# Interpolate Temperature onto the new grid
# Here we use 'linear' interpolation; you can also try 'cubic' or 'nearest'
temp_grid = griddata(points, temperature, grid_points, method='nearest')


# Reshape the temperature grid back to the 3D grid shape
temp_grid = temp_grid.reshape(grid_latitudes.size, grid_longitudes.size, grid_depths.size)


In [12]:
fig,ax = plt.subplots(nrows=1, figsize=(10,6))#, 
plt.scatter(df.Longitude, df.Latitude, color='red', label='Original Points', zorder=5, marker='o', s=100)
plt.scatter(grid_points[:, 0], grid_points[:, 1], color='blue', label='Grid Points', zorder=3, marker='x', s=30)

# Plot grid points (lat, lon)

In [13]:
# 3d plot
# Define the color range for the contour plot
kw = {
    'vmin': np.nanmin(temp_grid),
    'vmax': np.nanmax(temp_grid),
    'levels': np.linspace(np.nanmin(temp_grid), np.nanmax(temp_grid), 10),
}
# Create the 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

#top
_ = ax.contourf(
    grid_lon[:, :, 0], grid_lat[:, :, 0], temp_grid[:, :, 0],
    zdir='z', offset=grid_depth.min(), **kw
)

#bottom
_ = ax.contourf(
    grid_lon[:, :, -1], grid_lat[:, :, -1], temp_grid[:, :, -1], #grid_lat[:,:,-1] == grid_lat[:,:,1]?!
    zdir='z', offset=grid_depth.max(), **kw
)

#longitude-depth at lon_min
C = ax.contourf(
    grid_lon[0, :, :], temp_grid[0, :, :], grid_depth[0, :, :],
    zdir = 'y', 
    offset = grid_lat.min(), 
    **kw
)

#lon-depth at lon_max
_ = ax.contourf(
    grid_lon[-1, :, :], temp_grid[-1, :, :], grid_depth[-1, :, :],
    zdir = 'y', 
    offset = grid_lat.max(), 
    **kw
)

#lat-depth at lat.min
_ = ax.contourf(
    temp_grid[:, 0, :], grid_lat[:, 0, :], grid_depth[:, 0, :],
    zdir='x', offset=grid_lon.min(), **kw
)

# lon-depth at lat.max
_ = ax.contourf(
    temp_grid[:, -1, :], grid_lat[:, -1, :], grid_depth[:, -1, :],
    zdir='x', offset=grid_lon.max(), **kw
)



# Set limits of the plot from coord limits
xmin, xmax = grid_lon.min(), grid_lon.max()
ymin, ymax = grid_lat.min(), grid_lat.max()
zmin, zmax = grid_depth.max(), grid_depth.min()
ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

# Plot edges
edges_kw = dict(color='0.4', linewidth=1, zorder=1e3)
ax.plot([xmax, xmax], [ymin, ymax], [zmax, zmax], **edges_kw)
ax.plot([xmin, xmax], [ymin, ymin], [zmax, zmax], **edges_kw)
ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], **edges_kw)
ax.plot([xmin, xmax], [ymin, ymin], [zmin, zmin], **edges_kw)
ax.plot([xmin, xmin], [ymin, ymin], [zmin, zmax], **edges_kw)
ax.plot([xmax, xmax], [ymin, ymax], [zmin, zmin], **edges_kw)
ax.plot([xmax, xmax], [ymax, ymax], [zmin, zmax], **edges_kw)
ax.plot([xmin, xmax], [ymax, ymax], [zmax, zmax], **edges_kw)
ax.plot([xmin, xmin], [ymin, ymax], [zmax, zmax], **edges_kw)
ax.plot([xmin, xmin], [ymin, ymax], [zmin, zmin], **edges_kw)
ax.plot([xmin, xmin], [ymax, ymax], [zmin, zmax], **edges_kw)
ax.plot([xmin, xmax], [ymax, ymax], [zmin, zmin], **edges_kw)

# Set labels and ticks
ax.set(
    xlabel='Longitude [°]',
    ylabel='Latitude [°]',
    zlabel='Depth [cm]'
    # zticks=[-1, -200, -300],  # Customize these ticks as per your depth range
)

# Add colorbar
fig.colorbar(C, ax=ax, fraction=0.02, pad=0.1, label='Temperature [°C]')

# Show the plot
plt.show()


In [ ]:
fig,ax = plt.subplots(nrows=1, figsize=(10,6))#, 
plt.scatter(df.Longitude,df.Latitude, c = df['Temperature'], cmap = 'coolwarm')
cbar = plt.colorbar(cmap='coolwarm', pad=0.03)
cbar.set_label('CTD_Temp_SML', fontsize=18, fontweight='bold')
cbar.ax.tick_params(axis='y', which='both', labelsize=14, width=2)  # Adjust length and width as desired


In [173]:
fig,ax = plt.subplots(nrows=1, figsize=(10,6))#, 
plt.scatter(df.Longitude, df.Latitude, color='red', label='Original Points',  marker='o', s=100)
plt.scatter(grid_points[:, 0], grid_points[:, 1], color='blue', label='Grid Points', marker='x', s=30)



In [ ]:
# grouped = df.groupby('Depth')
 
# for depth, data in grouped:
#     plt.figure(figsize=(8, 6))
#     plt.scatter(data['Longitude'], data['Latitude'], label=f'Data for {depth}', alpha=0.6, c = data['Temperature'])
#     plt.title(f"Longitude vs Latitude for {depth}")
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.grid(True)
#     plt.legend()
#     plt.show()
